# Imports

In [2]:
# === Standard Library Imports ===
import csv
import glob
import itertools
import json
import os
import pickle
import random
import re
import time
from collections import defaultdict, deque
from datetime import datetime

# === Data Handling ===
import numpy as np
import pandas as pd

# === Utilities & Progress Bars ===
from tqdm import tqdm, trange

# === Evaluation Metrics ===
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    cohen_kappa_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score
)
from sklearn.utils.multiclass import unique_labels
from bert_score import score as bert_score

# === Machine Learning & Embeddings ===
import faiss
import torch
from sentence_transformers import SentenceTransformer

# === OpenAI & Azure OpenAI ===
import openai
import tiktoken
from openai import AzureOpenAI, OpenAIError, RateLimitError
from langchain_openai import AzureOpenAI as LangchainAzureOpenAI, AzureOpenAIEmbeddings

# === Google Gemini ===
import google.generativeai as genai

# === RAGAS Evaluation ===
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness
)

# === ChromaDB ===
import chromadb
from langchain_chroma import Chroma

# === PDF Handling ===
import pdfplumber
from PyPDF2 import PdfReader

# === Web & API Requests ===
import requests
from bs4 import BeautifulSoup  # pip install beautifulsoup4

# === Web Scraping (Selenium) ===
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# === Custom Modules ===
from LLM_functions import *
from datasets import Dataset

In [2]:
# Inicializar o cliente do Azure OpenAI Service com autenticação baseada em chave    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2025-01-01-preview",
)
# Configure Azure OpenAI Embeddings
embedding_function = AzureOpenAIEmbeddings(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    azure_deployment="TextEmbeddings",  # Ensure this is the correct deployment name in Azure
    openai_api_version="2025-01-01-preview",  # Match the API version
    max_retries=50
)

#### Load embeddings

In [3]:
db_gemini = Chroma(persist_directory="../../data/LLM/embedings/chroma_db_gemini")
db_minilm = Chroma(persist_directory="../../data/LLM/embedings/chroma_db_minilm")
db_openai = Chroma(persist_directory="../../data/LLM/embedings/chroma_db_openAI")


# Load stored embeddings
stored_gemini = load_all_embeddings(db_gemini)
stored_minilm = load_all_embeddings(db_minilm)
stored_openai = load_all_embeddings(db_openai)

# Print count of stored embeddings
print(f"Total Gemini embeddings: {len(stored_gemini['documents'])}")
print(f"Total MiniLM embeddings: {len(stored_minilm['documents'])}")
print(f"Total OpenAI embeddings: {len(stored_openai['documents'])}")

Total Gemini embeddings: 1327
Total MiniLM embeddings: 1837
Total OpenAI embeddings: 1327


In [4]:
# Retrieve a single stored embedding to check dimensions
sample_gemini = db_gemini._collection.get(include=["embeddings"], limit=1)
sample_minilm = db_minilm._collection.get(include=["embeddings"], limit=1)
sample_openai= db_openai._collection.get(include=["embeddings"], limit=1)

# Extract the first embedding from each (if available)
print("Gemini Embedding Dimensions:", len(sample_gemini["embeddings"][0]))
print("MiniLM Embedding Dimensions:", len(sample_minilm["embeddings"][0]))
print("Open AI Embedding Dimensions:", len(sample_openai["embeddings"][0]))

Gemini Embedding Dimensions: 768
MiniLM Embedding Dimensions: 384
Open AI Embedding Dimensions: 1536


# Evaluation

In [10]:
# Caminhos dos ficheiros
GROUND_TRUTH_PATH = "../../data/LLM/Lung_Cancer_Treatment_Stages_Final.csv"
INPUT_PATIENTS_PATH = "../../data/LLM/patient_inputs_sample.csv"

matched, unmatched = test_stage_matching()


========= Patient Stage Matching =========

[✔] Patient 0: TNM: T1a-N0-M0 | Type: Adenocarcinoma → Matched Stage: **IA**
[✔] Patient 1: TNM: T1b-N0-M0 | Type: Adenocarcinoma → Matched Stage: **IA**
[✔] Patient 2: TNM: T1c-N0-M0 | Type: Squamus Cell Carcinoma → Matched Stage: **IA**
[✔] Patient 3: TNM: T2a-N0-M0 | Type: Large cell → Matched Stage: **IB**
[✔] Patient 4: TNM: T2b-N0-M0 | Type: Squamus Cell Carcinoma → Matched Stage: **IIA**
[✔] Patient 5: TNM: T1a-N1-M0 | Type: Adenocarcinoma → Matched Stage: **IIB**
[✔] Patient 6: TNM: T1b-N1-M0 | Type: Adenocarcinoma → Matched Stage: **IIB**
[✔] Patient 7: TNM: T1c-N1-M0 | Type: Squamus Cell Carcinoma → Matched Stage: **IIB**
[✔] Patient 8: TNM: T2a-N1-M0 | Type: Squamus Cell Carcinoma → Matched Stage: **IIB**
[✔] Patient 9: TNM: T2b-N1-M0 | Type: Large cell → Matched Stage: **IIB**
[✔] Patient 10: TNM: T3-N0-M0 | Type: Adenocarcinoma → Matched Stage: **IIB**
[✔] Patient 11: TNM: T1a-N2-M0 | Type: Adenocarcinoma → Matched Stage: **IIIA

## Generate Responses - GPT

In [11]:
#  CONFIGURATION
N_REPEATS = 10
EMBEDDING_MODELS = ["gemini", "minilm", "openai"]
RETRIEVAL_METHODS = ["cosine", "bm25", "combined"]
LLM_MODELS = ["gpt-4o"]
OUTPUT_PATH = "../../data/LLM/GPT/repeated_evaluation_results.csv"
SUMMARY_PATH = "../../data/LLM/GPT/evaluation_summary.csv"
GROUPED_PATH = "../../data/LLM/GPT/retrieval_strategy_comparison.csv"


# Rate limit control
REQUEST_TIMESTAMPS = deque()
TOKENS_USED = 0
MAX_REQUESTS_PER_MINUTE = 2500
MAX_TOKENS_PER_MINUTE = 250000

## Generate Responses - Gemini

In [12]:
#  CONFIGURATION  #
N_REPEATS = 10
EMBEDDING_MODELS = ["gemini", "minilm", "openai"]
RETRIEVAL_METHODS = ["cosine", "bm25", "combined"]
LLM_MODELS = ["gemini"]
OUTPUT_PATH = "../../data/LLM/Evaluation/Gemini/repeated_evaluation_results.csv"
SUMMARY_PATH = "../../data/LLM/Evaluation/Gemini/evaluation_summary.csv"
GROUPED_PATH = "../../data/LLM/Evaluation/Gemini/retrieval_strategy_comparison.csv"

In [64]:
GPT = pd.read_csv('../../data/LLM/GPT/repeated_evaluation_results.csv')

In [65]:
GPT['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       1400
IIB         503
ES-SCLC     402
IIA         165
IVB         145
IA          119
IIIA        107
IV           90
IB           68
IIIC         59
I            54
LS-SCLC      53
IVA          25
II           20
III          19
IC            9
IIC           2
Name: count, dtype: int64

In [66]:
print(GPT["true_stage"].apply(type).value_counts())
print(GPT["predicted_stage"].apply(type).value_counts())

true_stage
<class 'str'>    3240
Name: count, dtype: int64
predicted_stage
<class 'str'>    3240
Name: count, dtype: int64


In [67]:
# Corrigir
GPT_ = correct_predicted_stages(GPT)

In [68]:
GPT_.columns

Index(['input_idx', 'run_idx', 'embedding_model', 'retrieval_method',
       'llm_model', 'T_stage', 'N_stage', 'M_stage', 'cancer_type',
       'true_stage', 'predicted_stage', 'ground_truth_answer',
       'predicted_treatment', 'raw_output', 'contexts', 'stage_match',
       'RAGAS_faithfulness', 'RAGAS_answer_relevancy',
       'RAGAS_context_precision', 'RAGAS_context_recall', 'bertscore_f1',
       'bleu', 'rougeL'],
      dtype='object')

In [69]:
GPT_['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       1410
IIB         526
ES-SCLC     402
IVB         197
IIA         173
IA          135
IIIA        115
IB           77
IIIC         59
LS-SCLC      53
IVA          47
IC           24
IV           12
IVC           4
I             3
IIC           3
Name: count, dtype: int64

In [70]:
GPT2 = pd.read_csv('../../data/LLM/GPT/repeated_evaluation_results_fixed.csv')

In [71]:
GPT2['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       1410
IIB         521
ES-SCLC     398
IVB         194
IIA         174
IA          134
IIIA        113
IB           78
IIIC         60
LS-SCLC      52
IVA          49
IC           22
IV           16
I             9
IVC           4
II            2
IIC           2
III           2
Name: count, dtype: int64

In [73]:
GPT_.to_csv('../../data/GPT_results.csv')

In [6]:
Gemnini = pd.read_csv('../../data/LLM/Gemini/repeated_evaluation_results.csv')

In [7]:
Gemnini['predicted_stage'].value_counts(dropna=False)

predicted_stage
III        879
ES-SCLC    384
IIA        337
IIIB       272
IIIA       258
NaN        182
IB         165
IIIC       137
IIB        134
IV         122
LS-SCLC     91
IVB         91
IA          63
IVA         53
I           34
II          21
IVC         17
Name: count, dtype: int64

In [8]:
print(Gemnini["true_stage"].apply(type).value_counts())
print(Gemnini["predicted_stage"].apply(type).value_counts())

true_stage
<class 'str'>    3240
Name: count, dtype: int64
predicted_stage
<class 'str'>      3058
<class 'float'>     182
Name: count, dtype: int64


In [9]:
# Corrigir
Gemnini_ = correct_predicted_stages(Gemnini)

In [10]:
Gemnini_.columns

Index(['input_idx', 'run_idx', 'embedding_model', 'retrieval_method',
       'llm_model', 'T_stage', 'N_stage', 'M_stage', 'cancer_type',
       'true_stage', 'predicted_stage', 'ground_truth_answer',
       'predicted_treatment', 'raw_output', 'contexts', 'stage_match'],
      dtype='object')

In [11]:
Gemnini_['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       801
IIIA       436
ES-SCLC    384
IIA        345
IIIC       319
IA         262
IB         174
IVB        165
IIB        152
LS-SCLC     91
IVA         72
IVC         38
I            1
Name: count, dtype: int64

In [12]:
Gemnini2 = pd.read_csv('../../data/LLM/Gemini/repeated_evaluation_results_fixed.csv')

In [13]:
Gemnini2['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       800
IIIA       447
ES-SCLC    375
IIA        347
IIIC       320
IA         266
IB         168
IIB        154
IVB        148
LS-SCLC     75
IVA         72
IV          29
IVC         29
III          9
V            1
Name: count, dtype: int64

In [15]:
Gemnini_.to_csv('../../data/Gemini_results.csv')

# Evaluate Responses

In [14]:
# Rate limit control
REQUEST_TIMESTAMPS = deque()
TOKENS_USED = 0
MAX_REQUESTS_PER_MINUTE = 3500
MAX_TOKENS_PER_MINUTE = 90000

## GPT Evaluation

In [18]:
evaluate_from_csv_only(
    input_csv_path = '../../data/LLM/Evaluation/GPT/GPT_results.csv', 
    output_path="../../data/LLM/Evaluation/Final_eval_GPT/repeated_evaluation_results.csv",
    grouped_output_path="../../data/LLM/Evaluation/Final_eval_GPT/results_summary.csv",
    metrics_output_path="../../data/LLM/Evaluation/Final_eval_GPT/metrics_and_confusion_matrix.csv",
    classification_report_output_path="../../data/LLM/Evaluation/Final_eval_GPT/classification_report.csv",
    confusion_matrices_csv_path="../../data/LLM/Evaluation/Final_eval_GPT/confusion_matrix_report.csv",
    ragas_output_path= "../../data/LLM/Evaluation/Final_eval_GPT/ragas_summary.csv"
    )

BERTScore batches:   0%|                                 | 0/33 [00:00<?, ?it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BERTScore batches:   3%|▊                        | 1/33 [01:37<51:59, 97.48s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BERTScore batches:   6%|█▌                       | 2/33 [03:03<46:49, 90.64s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weig


[✔] Computing RAGAS...


Evaluating:   0%|          | 0/1440 [00:01<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[328]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[452]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[632]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[776]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1059]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1196]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1298]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1311]: TimeoutError()


Evaluating:   0%|          | 0/1440 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/1440 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[303]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[326]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[376]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[403]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[555]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[556]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[560]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[704]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[944]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[951]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[999]: TimeoutError()


Evaluating:   0%|          | 0/1440 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[676]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1256]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1263]: TimeoutError()


[✔] Finished evaluation from CSV only.


## Gemini Evaluation

In [19]:
evaluate_from_csv_only(
    input_csv_path = "../../data/LLM/Evaluation/Gemini/Gemini_results.csv", 
    output_path="../../data/LLM/Evaluation/Final_eval_Gemini/repeated_evaluation_results.csv",
    grouped_output_path="../../data/LLM/Evaluation/Final_eval_Gemini/results_summary.csv",
    metrics_output_path="../../data/LLM/Evaluation/Final_eval_Gemini/metrics_and_confusion_matrix.csv",
    classification_report_output_path="../../data/LLM/Evaluation/Final_eval_Gemini/classification_report.csv",
    confusion_matrices_csv_path="../../data/LLM/Evaluation/Final_eval_Gemini/confusion_matrix_report.csv",
    ragas_output_path= "../../data/LLM/Evaluation/Final_eval_Gemini/ragas_summary.csv"
    )

BERTScore batches:   0%|                                 | 0/33 [00:00<?, ?it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BERTScore batches:   3%|▊                        | 1/33 [00:48<25:41, 48.18s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BERTScore batches:   6%|█▌                       | 2/33 [01:34<24:17, 47.01s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weig


[✔] Computing RAGAS...


Evaluating:   0%|          | 0/1440 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[112]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[140]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[368]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[380]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[679]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[744]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1064]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1078]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1140]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1203]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1306]: TimeoutError()


Evaluating:   0%|          | 0/1440 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/1440 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[256]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[263]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[268]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[319]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[495]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[496]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[575]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[734]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[974]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1334]: TimeoutError()


Evaluating:   0%|          | 0/1440 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[120]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[139]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[956]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1076]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1380]: TimeoutError()


[✔] Finished evaluation from CSV only.


# Fine tune Best results (TopK and Temperature)

In [ ]:
k_range = [5, 7, 10, 15]
temperature_range = [0, 0.3, 0.5, 0.7]
n_repeats = 10

## Setting 1:  GeminiCombinedGemini

In [42]:
configs_1 = [
    {
        "embedding_model": "gemini",
        "retrieval_method": "combined",
        "llm_model": "gemini"
    }
 
]

In [43]:
%%time
custom_evaluation_pipeline(
    configurations=configs_1,
    k_range=k_range,
    temperature_range=temperature_range,
    n_repeats=n_repeats,
    output_path="../../data/Evaluation/LLM/Final_eval/repeated_evaluation_results_GeminiCombinedGemini.csv",
    )

Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

...

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Patients: 100%|██████████████████████████| 36/36 [130:27:53<00:00, 13046.50s/it]


[✔] Saved raw results (before metrics) to /Users/catarinasilva/Desktop/LLM/Final_eval/repeated_evaluation_results_GeminiCombinedGemini.csv
CPU times: user 16min 54s, sys: 4min 19s, total: 21min 13s
Wall time: 5d 10h 27min 56s


## Setting 2: MiniLMCombinedGemini

In [25]:
configs_2 = [
    {
        "embedding_model": "openai",
        "retrieval_method": "combined",
        "llm_model": "gemini"
    }
 
]

In [26]:
%%time
custom_evaluation_pipeline(
    configurations=configs_2,
    k_range=k_range,
    temperature_range=temperature_range,
    n_repeats=n_repeats,
    output_path="../../data/LLM/Evaluation/Final_eval/OpenAICombinedGemini/repeated_evaluation_results_OpenAICombinedGemini.csv",
    )

Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

...

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Patients: 100%|██████████████████████████| 36/36 [129:37:26<00:00, 12962.40s/it]


[✔] Saved raw results (before metrics) to /Users/catarinasilva/Desktop/LLM/Final_eval/OpenAICombinedGemini/repeated_evaluation_results_OpenAICombinedGemini.csv
CPU times: user 16min 47s, sys: 4min 26s, total: 21min 14s
Wall time: 5d 9h 37min 29s


## Setting 3: OpenAIBM25Gemini

In [27]:
configs_3 = [
    {
        "embedding_model": "openai",
        "retrieval_method": "bm25",
        "llm_model": "gemini"
    }
 
]

In [28]:
%%time
custom_evaluation_pipeline(
    configurations=configs_3,
    k_range=k_range,
    temperature_range=temperature_range,
    n_repeats=n_repeats,
    output_path="../../data/LLM/Evaluation/Final_eval/repeated_evaluation_results_OpenAIBM25Gemini.csv",
    )

Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

...

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Configurations:   0%|                                     | 0/1 [00:00<?, ?it/s]

[🚨] Gemini quota exceeded. Sleeping for 24 hours.
[⏳ Attempt 1/2] Retrying in 15 seconds...



Patients: 100%|██████████████████████████| 36/36 [153:47:20<00:00, 15378.90s/it]


[✔] Saved raw results (before metrics) to /Users/catarinasilva/Desktop/LLM/Final_eval/repeated_evaluation_results_OpenAIBM25Gemini.csv
CPU times: user 15min 41s, sys: 4min 16s, total: 19min 57s
Wall time: 6d 9h 47min 23s


##### GeminiCombinedGemini

In [3]:
df_GeminiCombinedGemini = pd.read_csv('../../data/LLM/Final_eval/GeminiCombinedGemini/repeated_evaluation_results_GeminiCombinedGemini.csv')

In [5]:
df_GeminiCombinedGemini.shape

(5760, 18)

In [11]:
df_GeminiCombinedGemini['predicted_stage'].value_counts(dropna=False)

predicted_stage
III        1548
ES-SCLC     658
IIA         591
IIIB        515
IIIA        371
NaN         324
IB          281
IIB         280
LS-SCLC     238
IV          228
IIIC        218
IVB         170
IA          131
IVA          94
I            67
IVC          30
II           16
Name: count, dtype: int64

In [7]:
print(df_GeminiCombinedGemini["true_stage"].apply(type).value_counts())
print(df_GeminiCombinedGemini["predicted_stage"].apply(type).value_counts())

true_stage
<class 'str'>    5760
Name: count, dtype: int64
predicted_stage
<class 'str'>      5436
<class 'float'>     324
Name: count, dtype: int64


In [44]:
# Corrigir
df_GeminiCombinedGemini_corrigido = correct_predicted_stages(df_GeminiCombinedGemini)

In [33]:
df_GeminiCombinedGemini_corrigido.columns

Index(['embedding_model', 'retrieval_method', 'llm_model', 'top_k',
       'temperature', 'input_idx', 'run_idx', 'T_stage', 'N_stage', 'M_stage',
       'cancer_type', 'true_stage', 'predicted_stage', 'ground_truth_answer',
       'predicted_treatment', 'raw_output', 'contexts', 'stage_match'],
      dtype='object')

In [45]:
df_GeminiCombinedGemini_corrigido['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       1434
IIIA        660
ES-SCLC     658
IIA         603
IIIC        582
IA          491
IIB         304
IB          295
IVB         291
LS-SCLC     238
IVA         120
IVC          84
Name: count, dtype: int64

In [ ]:
df_GeminiCombinedGemini_corrigido.to_csv('../../data/LLM/Final_eval/GeminiCombinedGemini/repeated_evaluation_results_GeminiCombinedGemini_fixed.csv')

In [46]:
results_1 = df_GeminiCombinedGemini[df_GeminiCombinedGemini['predicted_stage']!=df_GeminiCombinedGemini['true_stage']][['predicted_treatment','predicted_stage', 'true_stage']]

In [13]:
results_1.shape

(4055, 3)

In [16]:
results_1['predicted_stage'].value_counts(dropna=False)

predicted_stage
III        1548
IIA         576
ES-SCLC     338
NaN         324
IIIB        254
IV          228
IVB         170
IB          165
IIIC        114
LS-SCLC      96
I            67
IIIA         58
IA           39
IIB          32
IVC          30
II           16
Name: count, dtype: int64

In [47]:
results_2 = df_GeminiCombinedGemini_corrigido[df_GeminiCombinedGemini_corrigido['predicted_stage']!=df_GeminiCombinedGemini_corrigido['true_stage']][['predicted_treatment','predicted_stage', 'true_stage']]

In [48]:
results_2.shape

(2777, 3)

In [49]:
results_2['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       700
IIA        588
ES-SCLC    338
IIIC       325
IVB        291
IB         173
IIIA       110
LS-SCLC     96
IVC         84
IA          39
IIB         33
Name: count, dtype: int64

##### OpenAICombinedGemini

In [3]:
df_OpenAICombinedGemini = pd.read_csv('../../data/LLM/Final_eval/OpenAICombinedGemini/repeated_evaluation_results_OpenAICombinedGemini.csv')

In [5]:
df_OpenAICombinedGemini.shape

(5760, 18)

In [11]:
df_OpenAICombinedGemini['predicted_stage'].value_counts(dropna=False)

predicted_stage
III        1548
ES-SCLC     658
IIA         591
IIIB        515
IIIA        371
NaN         324
IB          281
IIB         280
LS-SCLC     238
IV          228
IIIC        218
IVB         170
IA          131
IVA          94
I            67
IVC          30
II           16
Name: count, dtype: int64

In [7]:
print(df_OpenAICombinedGemini["true_stage"].apply(type).value_counts())
print(df_OpenAICombinedGemini["predicted_stage"].apply(type).value_counts())

true_stage
<class 'str'>    5760
Name: count, dtype: int64
predicted_stage
<class 'str'>      5436
<class 'float'>     324
Name: count, dtype: int64


In [44]:
# Corrigir
df_OpenAICombinedGemini_corrigido = correct_predicted_stages(df_OpenAICombinedGemini)

In [33]:
df_OpenAICombinedGemini_corrigido.columns

Index(['embedding_model', 'retrieval_method', 'llm_model', 'top_k',
       'temperature', 'input_idx', 'run_idx', 'T_stage', 'N_stage', 'M_stage',
       'cancer_type', 'true_stage', 'predicted_stage', 'ground_truth_answer',
       'predicted_treatment', 'raw_output', 'contexts', 'stage_match'],
      dtype='object')

In [45]:
df_OpenAICombinedGemini_corrigido['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       1434
IIIA        660
ES-SCLC     658
IIA         603
IIIC        582
IA          491
IIB         304
IB          295
IVB         291
LS-SCLC     238
IVA         120
IVC          84
Name: count, dtype: int64

In [ ]:
df_OpenAICombinedGemini_corrigido.to_csv('../../data/LLM/Final_eval/OpenAICombinedGemini/repeated_evaluation_results_OpenAICombinedGemini_fixed.csv')

In [46]:
results_3 = df_OpenAICombinedGemini[df_OpenAICombinedGemini['predicted_stage']!=df_OpenAICombinedGemini['true_stage']][['predicted_treatment','predicted_stage', 'true_stage']]

In [13]:
results_3.shape

(4055, 3)

In [16]:
results_3['predicted_stage'].value_counts(dropna=False)

predicted_stage
III        1548
IIA         576
ES-SCLC     338
NaN         324
IIIB        254
IV          228
IVB         170
IB          165
IIIC        114
LS-SCLC      96
I            67
IIIA         58
IA           39
IIB          32
IVC          30
II           16
Name: count, dtype: int64

In [47]:
results_4 = df_OpenAICombinedGemini_corrigido[df_OpenAICombinedGemini_corrigido['predicted_stage']!=df_OpenAICombinedGemini_corrigido['true_stage']][['predicted_treatment','predicted_stage', 'true_stage']]

In [48]:
results_4.shape

(2777, 3)

In [49]:
results_4['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       700
IIA        588
ES-SCLC    338
IIIC       325
IVB        291
IB         173
IIIA       110
LS-SCLC     96
IVC         84
IA          39
IIB         33
Name: count, dtype: int64

##### OpenAIBM25Gemini

In [29]:
df_OpenAIBM25Gemini = pd.read_csv('../../data/LLM/Final_eval/OpenAIBM25Gemini/repeated_evaluation_results_OpenAIBM25Gemini.csv')

In [30]:
df_OpenAIBM25Gemini.shape

(5760, 18)

In [31]:
df_OpenAIBM25Gemini['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       1468
ES-SCLC     660
IIIA        645
IIA         582
IIIC        533
IA          500
IIB         327
IVB         294
IB          286
LS-SCLC     250
IVA         124
IVC          91
Name: count, dtype: int64

In [32]:
print(df_OpenAIBM25Gemini["true_stage"].apply(type).value_counts())
print(df_OpenAIBM25Gemini["predicted_stage"].apply(type).value_counts())

true_stage
<class 'str'>    5760
Name: count, dtype: int64
predicted_stage
<class 'str'>    5760
Name: count, dtype: int64


In [33]:
# Corrigir
df_OpenAIBM25Gemini_corrigido = correct_predicted_stages(df_OpenAIBM25Gemini)

In [34]:
df_OpenAIBM25Gemini_corrigido.columns

Index(['embedding_model', 'retrieval_method', 'llm_model', 'top_k',
       'temperature', 'input_idx', 'run_idx', 'T_stage', 'N_stage', 'M_stage',
       'cancer_type', 'true_stage', 'predicted_stage', 'ground_truth_answer',
       'predicted_treatment', 'raw_output', 'contexts', 'stage_match'],
      dtype='object')

In [35]:
df_OpenAIBM25Gemini_corrigido['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       1468
ES-SCLC     660
IIIA        645
IIA         582
IIIC        533
IA          500
IIB         327
IVB         294
IB          286
LS-SCLC     250
IVA         124
IVC          91
Name: count, dtype: int64

In [36]:
df_OpenAIBM25Gemini_corrigido.to_csv('../../data/LLM/Final_eval/OpenAIBM25Gemini/repeated_evaluation_results_OpenAIBM25Gemini_fixed.csv')

In [46]:
results_5 = df_OpenAIBM25Gemini[df_OpenAIBM25Gemini['predicted_stage']!=df_OpenAIBM25Gemini['true_stage']][['predicted_treatment','predicted_stage', 'true_stage']]

In [13]:
results_5.shape

(4055, 3)

In [16]:
results_5['predicted_stage'].value_counts(dropna=False)

predicted_stage
III        1548
IIA         576
ES-SCLC     338
NaN         324
IIIB        254
IV          228
IVB         170
IB          165
IIIC        114
LS-SCLC      96
I            67
IIIA         58
IA           39
IIB          32
IVC          30
II           16
Name: count, dtype: int64

In [47]:
results_6 = df_OpenAIBM25Gemini_corrigido[df_OpenAIBM25Gemini_corrigido['predicted_stage']!=df_OpenAIBM25Gemini_corrigido['true_stage']][['predicted_treatment','predicted_stage', 'true_stage']]

In [48]:
results_6.shape

(2777, 3)

In [49]:
results_6['predicted_stage'].value_counts(dropna=False)

predicted_stage
IIIB       700
IIA        588
ES-SCLC    338
IIIC       325
IVB        291
IB         173
IIIA       110
LS-SCLC     96
IVC         84
IA          39
IIB         33
Name: count, dtype: int64

## Evaluate Best Configurations

In [18]:
REQUEST_TIMESTAMPS = deque()
TOKENS_USED = 0
MAX_REQUESTS_PER_MINUTE = 2500
MAX_TOKENS_PER_MINUTE = 250000

### GeminiCombinedGemini

In [35]:
evaluate_from_csv(
    input_csv_path = "../../data/LLM/Evaluation/Final_eval/GeminiCombinedGemini/repeated_evaluation_results_GeminiCombinedGemini_fixed.csv", 
    output_path="../../data/LLM/Evaluation/Final_eval/GeminiCombinedGemini/repeated_evaluation_results.csv",
    grouped_output_path="../../data/LLM/Evaluation/Final_eval/GeminiCombinedGemini/results_summary.csv",
    metrics_output_path="/../../data/LLM/Evaluation/Final_eval/GeminiCombinedGemini/metrics_and_confusion_matrix.csv",
    classification_report_output_path="../../data/LLM/Evaluation/Final_eval/GeminiCombinedGemini/classification_report.csv",
    confusion_matrices_csv_path="../../data/LLM/Evaluation/Final_eval/GeminiCombinedGemini/confusion_matrix_report.csv",
    ragas_output_path= "../../data/LLM/Evaluation/Final_eval/GeminiCombinedGemini/ragas_summary.csv"
    )


[✔] Computing RAGAS...


RAGAS Evaluation:   0%|                                  | 0/16 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[31]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[44]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[48]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[59]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[60]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[64]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[100]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[103]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[159]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[175]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[180]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[204]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[220]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[232]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[296]: TimeoutError()
ERROR:ragas.exec

Evaluating:   0%|          | 0/240 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[0]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[23]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[63]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[79]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[100]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[116]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[154]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[211]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[215]: TimeoutError()
RAGAS Evaluation: 100%|████████████████████| 16/16 [19:01:24<00:00, 4280.27s/it]

[✔] Finished RAGAS evaluation safely.


### OpenAICombinedGemini

In [19]:
evaluate_from_csv(
    input_csv_path = "../../data/LLM/Evaluation/Final_eval/OpenAICombinedGemini/repeated_evaluation_results_OpenAICombinedGemini_fixed.csv", 
    output_path="../../data/LLM/Evaluation/Final_eval/OpenAICombinedGemini/repeated_evaluation_results.csv",
    grouped_output_path="../../data/LLM/Evaluation/Final_eval/OpenAICombinedGemini/results_summary.csv",
    metrics_output_path="/../../data/LLM/Evaluation/Final_eval/OpenAICombinedGemini/metrics_and_confusion_matrix.csv",
    classification_report_output_path="../../data/LLM/Evaluation/Final_eval/OpenAICombinedGemini/classification_report.csv",
    confusion_matrices_csv_path="../../data/LLM/Evaluation/Final_eval/OpenAICombinedGemini/confusion_matrix_report.csv",
    ragas_output_path= "../../data/LLM/Evaluation/Final_eval/OpenAICombinedGemini/ragas_summary.csv"
    )

BERTScore batches:   0%|                                 | 0/58 [00:00<?, ?it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BERTScore batches:   2%|▍                        | 1/58 [00:51<49:21, 51.95s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BERTScore batches:   3%|▊                        | 2/58 [01:43<48:10, 51.61s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weig


[✔] Computing RAGAS...


RAGAS Evaluation:   0%|                                  | 0/16 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[135]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[214]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[230]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[240]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[247]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[252]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[256]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[260]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[280]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[282]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[287]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[292]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[319]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[324]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[331]: TimeoutError()
ERROR:raga

Evaluating:   0%|          | 0/240 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[30]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[102]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[168]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[187]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[194]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[206]: TimeoutError()
RAGAS Evaluation: 100%|████████████████████| 16/16 [19:13:39<00:00, 4326.24s/it]

[✔] Finished RAGAS evaluation safely.


### OpenAIBM25Gemini

In [20]:
evaluate_from_csv(
    input_csv_path = "../../data/LLM/Evaluation/Final_eval/OpenAIBM25Gemini/repeated_evaluation_results_OpenAIBM25Gemini_fixed.csv", 
    output_path="../../data/LLM/Evaluation/Final_eval/OpenAIBM25Gemini/repeated_evaluation_results.csv",
    grouped_output_path="../../data/LLM/Evaluation/Final_eval/OpenAIBM25Gemini/results_summary.csv",
    metrics_output_path="/../../data/LLM/Evaluation/Final_eval/OpenAIBM25Gemini/metrics_and_confusion_matrix.csv",
    classification_report_output_path="../../data/LLM/Evaluation/Final_eval/OpenAIBM25Gemini/classification_report.csv",
    confusion_matrices_csv_path="../../data/LLM/Evaluation/Final_eval/OpenAIBM25Gemini/confusion_matrix_report.csv",
    ragas_output_path= "../../data/LLM/Evaluation/Final_eval/OpenAIBM25Gemini/ragas_summary.csv"
    )

BERTScore batches:   0%|                                 | 0/58 [00:00<?, ?it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BERTScore batches:   2%|▍                        | 1/58 [00:49<47:12, 49.69s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BERTScore batches:   3%|▊                        | 2/58 [01:36<44:43, 47.92s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weig


[✔] Computing RAGAS...


RAGAS Evaluation:   0%|                                  | 0/16 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[196]: TimeoutError()


...

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[36]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[124]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[132]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[135]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[151]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[152]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[160]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[192]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[204]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[208]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[228]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[244]: TimeoutError()


Evaluating:   0%|          | 0/240 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[71]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[132]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[212]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[228]: TimeoutError()
RAGAS Evaluation: 100%|████████████████████| 16/16 [19:16:39<00:00, 4337.44s/it]

[✔] Finished RAGAS evaluation safely.
